## Applying MaxEnt to explore the inner working of GPT2 

Let $\vec e_1, \vec e_2, \dots, \vec e_{n+1}$ be the tokens in the input prompt. We'd like to determine:
$$
p(\vec x | \vec e_1, \vec e_2, \dots, \vec e_n)
$$
which is essentially what the transformer computes after $12$ repetitions of the ATTN+FFNN mechanism. In principle, since the action of the transformer (_the dynamical rule_) is known, one could potentially figure out exactly the resulting pdf in the vocabulary space. This is however analytically infeasible, since the transformer is made of several non linear layers (FFNN) and the dynamics is allegedely extremely complicated. Thus, analoguosly to what we do in stat mech, we hope our system is ergodic and try to build a statistical description on top of the whole embedding space.

 In particular, we resort to the __MaxEnt principle__ to compute (analytically or not) the final conditioned pdf. Note that we do have the "correct" pdf $p(\vec x | \vec e_1, \vec e_2, \dots, \vec e_n)$. We just want to see whether, upon applying MaxEnt with the right constraints, we can obtain something similar to the distribution computed by the transformer

 ### Bolztmann-like approach
 The entropy functional on reads:

$$
S[p] = -k \int d^D x \> p(\vec x) \ln p(\vec x) 
$$
the first constraint is, obviously, the normalization $ \int d^D x \> p(\vec x) = 1 $

In this first paragraph, we apply the following constraint:
$$
\langle \vec x \rangle = \vec{e}_{n+1}
$$
because we do have the "right" next token (again, we don't want to perform prediction here with the maxEnt. We just wish to see if the resulting pdf has some similarities with the one suggested by the maximization of $S$).

This leads to an analytic form for $p(\vec x | \vec e_1, \vec e_2, \dots, \vec e_n)$:
$$
p(\vec x | \vec e_1, \vec e_2, \dots, \vec e_n) = p(\vec x) = \frac{1}{Z} \exp(\vec \mu \cdot \vec x)
$$
with $Z =\prod_{i=1}^{D} \frac{2 sinh(\mu_i)}{\mu_i}$ and the vector $\mu$ (the lagrange multiplier) is such that:
$$
\vec e_{n+1} = -\frac{1}{\vec\mu} + \frac{1}{\tanh(\vec\mu)} \Longleftrightarrow \vec\mu = f(\vec e_{n+1})
$$

Practically speaking, we have built a pdf Boltzmann-like where the expected value has to be the correct next token and the mode (the vector in the embedding space that maximizes the probability) is precisely $\vec \mu$. Let's see what $\vec \mu$ is

In [1]:
### IMPORT AND CONFIGURATION

import numpy as np
import torch
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
import matplotlib.pyplot as plt
import seaborn as sns


tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
vocab = tokenizer.get_vocab()
model = GPT2Model.from_pretrained("gpt2", output_hidden_states=True, output_attentions=True)
embeddings = model.wte.weight.data

def get_tokens_prob(x, k=5):
    # Compute the similarity between the token and all the tokens in the vocabulary, then applies softmax
    prob = torch.softmax(torch.matmul(x,embeddings.T), dim=-1)
    # Get the top k tokens
    top = torch.topk(prob, k=k)
    idxs = top.indices
    tokens = [tokenizer.decode(idx) for idx in idxs]
    return top.values, idxs, tokens

In [2]:
# INFERENCE STEP

# The prompt
prompt = "I'm an Italian physicist who lived in Padua. I am known for my work in the field of optics and astronomy. " \
        "I was born in 1564 and died in 1642. My name is Galileo"
inputs = tokenizer(prompt, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    hidden_states = outputs.hidden_states
layers = hidden_states[1:]
 
# Next output token prediction
nextToken = (layers[-1][0])[-1]
# Given this representation of the next token, we project it into the vocabulary space and build a pdf on top of it using softmax
lengthQueue = 10
prob, idxs, tokens = get_tokens_prob(nextToken, lengthQueue)
print("Next token prediction:")
for p, idx, token in zip(prob, idxs, tokens):
    print(f" -> logp = {-torch.log(p):.2f}: ['{token}'], idx = {idx}")

`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


Next token prediction:
 -> logp = 0.28: [' Galile'], idx = 32422
 -> logp = 2.29: ['.'], idx = 13
 -> logp = 3.12: [' and'], idx = 290
 -> logp = 3.55: [','], idx = 11
 -> logp = 5.17: ['."'], idx = 526
 -> logp = 5.62: [' ('], idx = 357
 -> logp = 5.87: [' Galileo'], idx = 45860
 -> logp = 5.99: [' in'], idx = 287
 -> logp = 6.42: [' I'], idx = 314
 -> logp = 6.75: [' de'], idx = 390


In [3]:
from scipy.optimize import fsolve

#This is the function we want to solve
def f(x,k):
    return 1 / np.tanh(x) - 1 / x - k

# We need to constraint the mean value of the distribution to be equal to the expected value of the token
tokenExpected = embeddings[32422]      # extracted from above

mu = torch.tensor(np.zeros(embeddings.shape[1]), dtype=torch.float32)
for i,d in enumerate(tokenExpected):
    xi = d.item()
    mu_i = fsolve(f, 1, args=(xi))
    mu[i] = (mu_i.item())

print(f"Succesfully computed mu vector.")
print("It represents the direction of maximum probability; projecting mu over the vocabolary we get: \n")
# mu should be the direction of maximum probability (mode). At what does it point to?
prob, idxs, tokens = get_tokens_prob(mu, 3)
for p, idx, token in zip(prob, idxs, tokens):
    print(f" -> logp = {-torch.log(p):.2f}: ['{token}'], idx = {idx}")

print("\nAs expected, it points practically with certainty to the right expected token")

Succesfully computed mu vector.
It represents the direction of maximum probability; projecting mu over the vocabolary we get: 

 -> logp = -0.00: [' Galile'], idx = 32422
 -> logp = 38.92: [' Canaver'], idx = 46858
 -> logp = 40.84: ['Palest'], idx = 32570

As expected, it points practically with certainty to the right expected token


As expected, $\vec \mu$ (the mode, determined with MaxEnt) is essentially identical to the mean value (that we imposed as the right token)



DA AGGIUNGERE: LOG-LIKELI ESPONENZIALE COL PRODOTTO SCALARE, DA AGGIUNGERE? GIUSTIFICAZIONE A POSTERIORI


### Using the attention matrix

The previous approach was clearly extremely simple and trivial, lacking precious information that we have on the resulting pdf (in fact, we didn't use the previous tokens $\{e_i\}$). Let us think of a new constraint. Since the dot product in the embedding space (upon normalization) is somehow related to the similarities between tokens, we may wish to constrain:
$$ 
\langle \vec x \cdot \vec e_i \rangle = something, \>\> \forall \vec e_i
$$
Since the transfomer provides us with attention matrices, why don't use those quantities to constrain our entropy. In particular, if $\hat A$ is the $n+1 \times n+1$ attention matrix at the end of 

In [45]:
# INFERENCE STEP. HERE, WE LET GPT2 PREDICT THE NEXT TOKEN. WE WILL USE PART OF THE RESULT TO PERFORM OUR STATISTICAL MAXENT ANALYSIS

# The prompt
prompt = "I'm a German-born theoretical physicist who revolutionized modern physics with my theory of relativity. " \
"I was awarded the Nobel Prize in Physics in 1921 for my explanation of the photoelectric effect." \
" I was born in 1879 and died in 1955. My name is Albert Einstein"


inputs = tokenizer(' '.join(prompt.split()[:-1]), return_tensors="pt")


with torch.no_grad():
    outputs = model(**inputs)
    hidden_states = outputs.hidden_states
layers = hidden_states[1:]

# attentions is a tuple of tensors. Each tensor is 12 attention maps of 16x16 matrices.
attentions = outputs.attentions

n = len(inputs["input_ids"][0])    # total number of tokens in the input

nextToken = (layers[-1][0])[-1]
prob, idxs, tokens = get_tokens_prob(nextToken, lengthQueue)
print("Masking the last token (Einstein), GPT2 would have predicted:")
for p, idx, token in zip(prob, idxs, tokens):
    print(f" -> logp = {-torch.log(p):.2f}: ['{token}'], idx = {idx}")


Masking the last token (Einstein), GPT2 would have predicted:
 -> logp = 0.58: [' Einstein'], idx = 24572
 -> logp = 4.39: [' B'], idx = 347
 -> logp = 4.67: [' von'], idx = 18042
 -> logp = 4.79: [' Hof'], idx = 37745
 -> logp = 4.83: ['.'], idx = 13
 -> logp = 4.86: [' H'], idx = 367
 -> logp = 4.94: [' Z'], idx = 1168
 -> logp = 5.00: [' J'], idx = 449
 -> logp = 5.07: [' Sch'], idx = 3059
 -> logp = 5.08: [' A'], idx = 317


In [277]:
lastStepAttentionMatrices = attentions[0][0]
vectorA = torch.tensor(np.zeros(n-1))              # Not the last token (itself w/ itself)
#iterate on the heads
for head in range(12):
    attentionHead = lastStepAttentionMatrices[head]
    column = attentionHead[-1,:]
    column = column[:-1]
    vectorA = (vectorA + column)

T = torch.nn.functional.one_hot(inputs.input_ids[0], num_classes=embeddings.shape[0]).float()
E = torch.matmul(T,embeddings)

In [ ]:
import numpy as np
from scipy.optimize import fsolve, least_squares

# f function (to improve stability, Taylor expansion around 0)
def f(z):
    z = np.asarray(z) 
    tol = 0.1
    result = np.empty_like(z, dtype=float)

    small_mask = np.abs(z) < tol
    z_small = z[small_mask]
    result[small_mask] = (-1/3)*z_small + (2/45)*z_small**3 - (17/945)*z_small**5

    # Maschera per valori "grandi" (|z| >= tol)
    big_mask = ~small_mask
    z_big = z[big_mask]
    result[big_mask] = - 1 / np.tanh(z_big) + 1 / z_big

    return result

def system(x, E, A):
    E = E[:-1, :]
    z = np.dot(E.T, x)     
    fz = f(z)               
    fz = fz.reshape(-1, 1)     
    v = E @ fz
    v = v.reshape(-1)      
    return v - A       


x0 = np.random.random(n-1)      # incredibly sensible to the initial condition
x0 = x0 / np.linalg.norm(x0)       # normalizing the initial condition
amplificationAttention = 1000      # PERCHÉ AMPLIFICARE AIUTA A STABILIZZARE? DA CAPIRE BENE CHE MATRICI METTERE...
mu = fsolve(system, x0, args=(E.numpy(), amplificationAttention*vectorA.numpy()))

# given mu, we can compute the expected value according to the maxent model
x_expected = f((E[:-1, :]).T @ mu)


print(' '.join(prompt.split()[:-1]))
prob, idxs, tokens = get_tokens_prob(torch.tensor(x_expected, dtype = torch.float), 10)
for p, _, token in zip(prob, idxs, tokens):
    print(f" -> {-torch.log(p):.2f}: '{token}'")

I'm a German-born theoretical physicist who revolutionized modern physics with my theory of relativity. I was awarded the Nobel Prize in Physics in 1921 for my explanation of the photoelectric effect. I was born in 1879 and died in 1955. My name is Albert
 -> 0.81: ' mathemat'
 -> 0.95: ' physicist'
 -> 1.93: ' physicists'
 -> 4.26: ' Physics'
 -> 5.79: ' Nobel'
 -> 5.97: ' relativity'
 -> 8.43: ' Einstein'
 -> 9.69: ' physics'
 -> 9.88: ' theorist'
 -> 10.42: ' mathematician'


/tmp/ipykernel_5027/3505794128.py:34: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  mu = fsolve(system, x0, args=(E.numpy(), amplificationAttention*vectorA.numpy()))
